In [1]:
import random
import time
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from tqdm import tqdm
import gc
from pyDOE import lhs
import time

# Загрузка данных
data = pd.read_csv('houston_short_300.csv')
# data = data.sample(n=300, random_state=42)
# data.to_csv('houston_short_300.csv', index=False)
X = data[['Latitude', 'Longitude', 'Year Built', 'Beds', 'Baths', 'buildingSize', 'lotSize', 'PostalCode']]
y = data['Price']

In [2]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
import pandas as pd
import time

# Функция для оптимизации
def objective(trial):
    # Определяем гиперпараметры, которые будет оптимизировать Optuna
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'num_leaves': trial.suggest_int('num_leaves', 2, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_bin': trial.suggest_int('max_bin', 10, 1000),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    # Модель LightGBM
    model = lgb.LGBMRegressor(**params)

    # Кросс-валидация с 3 фолдами
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error')
    mse = -scores.mean()  # Минимизируем MSE
    
    return mse

# Количество итераций для поиска гиперпараметров
n_trials = 500

# Запуск оптимизации с Optuna
start_time = time.time()
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

# Время до завершения оптимизации
end_time = time.time()
elapsed_time = end_time - start_time

# Получаем лучшее значение MSE и гиперпараметры
best_mse = study.best_value
best_trial = study.best_trial.number

# Сохранение результатов
results = {
    'Лучшее значение MSE': [best_mse],
    'Лучшая итерация': [best_trial],
    'Время до лучшей итерации (секунды)': [elapsed_time]
}

# Преобразование результатов в DataFrame и сохранение в CSV
df_results = pd.DataFrame(results)
df_results.to_csv('opt_houses.csv', index=False)

print("Лучшие гиперпараметры:")
print(study.best_params)
print(f"Лучшее значение MSE: {best_mse}")
print(f"Лучшая итерация: {best_trial}")
print(f"Время до лучшей итерации: {elapsed_time:.2f} секунд")


[I 2024-10-24 20:15:05,141] A new study created in memory with name: no-name-f873fd2e-27b6-4e05-aeb5-d42714c690d3
[I 2024-10-24 20:15:05,742] Trial 0 finished with value: 648690.4145324784 and parameters: {'n_estimators': 1356, 'max_depth': 3, 'num_leaves': 750, 'learning_rate': 0.06545988154023698, 'max_bin': 769, 'colsample_bytree': 0.8529865139608589}. Best is trial 0 with value: 648690.4145324784.
[I 2024-10-24 20:15:07,795] Trial 1 finished with value: 614742.1868486997 and parameters: {'n_estimators': 1935, 'max_depth': 80, 'num_leaves': 735, 'learning_rate': 0.08879241744710417, 'max_bin': 585, 'colsample_bytree': 0.6354060871295117}. Best is trial 1 with value: 614742.1868486997.
[I 2024-10-24 20:15:08,076] Trial 2 finished with value: 606250.5283629448 and parameters: {'n_estimators': 445, 'max_depth': 68, 'num_leaves': 57, 'learning_rate': 0.21442559206765097, 'max_bin': 595, 'colsample_bytree': 0.7599828007836207}. Best is trial 2 with value: 606250.5283629448.
[I 2024-10-24

Лучшие гиперпараметры:
{'n_estimators': 368, 'max_depth': 98, 'num_leaves': 77, 'learning_rate': 0.04145029763815322, 'max_bin': 166, 'colsample_bytree': 0.7269996697540853}
Лучшее значение MSE: 501628.5485473072
Лучшая итерация: 231
Время до лучшей итерации: 202.90 секунд
